In [34]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from uuid import uuid4
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from rag import textbookRAG

In [35]:
from test_metadata import metadata


In [36]:
book_dir = Path("../utilities/book")

In [37]:
rag = textbookRAG(metadata=metadata, book_dir=book_dir)
# rag.extract_from_metadata()

/Users/bryanmelvison/Code/FYP/KGV-AI-Tutor/backend/app/utilities/rag.py:54: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_store = Chroma(


In [38]:
rag.text_splitter._chunk_size

2000

In [39]:
len(rag.vector_store.get()["ids"])

384

In [45]:
rag.vector_store.get()

{'ids': ['6aec34aa-a9aa-4b2c-85d8-6719d1ae6035',
  'a3e2f1be-7810-4bfd-ac76-f72505beb1fb',
  'c31d637d-745b-44ea-8c2b-b9eea4fdcbd1',
  'c26ae4d7-0e9d-4f82-b356-f0487a9c6e43',
  '2f21ee24-3ca1-40e3-9ace-c43d0ba15b44',
  'e2f28b8e-a3e3-41c9-875c-a4e4e7614e30',
  '91d78b57-ced7-4a9d-90c2-020706e61fba',
  '191c8fd9-634b-400b-9c9a-260661d32a49',
  '0ca13a24-7a5d-463d-9397-b59eed319a32',
  'f2ffa860-a21a-42ab-bcc4-c88c12193884',
  '165267f9-69e7-4ef0-9873-612ff2d83d45',
  '9ef0140a-f282-44fe-a788-432db6bbee25',
  '221504de-cfdf-4080-b5aa-b04bb1b9e72a',
  'bf36b123-3abe-48c0-a7c0-b347da877157',
  'de44e07b-f53d-4d9c-a17d-6b4e70c14428',
  'cab78a6a-a907-4438-803b-5b364cbcabc3',
  '0a2beb32-8487-4e81-8e0b-31adeffeeebe',
  'ed444cac-c5df-4459-b315-c94bc898f44d',
  'efdd723c-23fa-4322-a8a9-26b1d16e4873',
  '00728c22-25af-4fd2-83ed-a10731d2d3b6',
  'f0ded2bb-66b7-4c34-8d1e-c39fc446923a',
  '1f52d472-ae06-445e-b19e-01763e2e2658',
  '32eafff5-197f-4a3d-aa0c-3b0cbf0ac156',
  '7d6ba569-601a-46d2-b52e-

In [43]:
a = rag.search("Can you tell me about sex?", subject = "Biology", chapter= 1)

In [44]:
a

"### KEY POINT\nOsmosis in cells is the net movement of water from a dilute solution to a more concentrated solution across the partially permeable cell membrane.\n\n## SPECIALISED EXCHANGE SURFACES\n\nAll cells exchange substances with their surroundings, but some parts of animals or plants are specially adapted for the exchange of materials because they have a very large surface area in proportion to their volume. In animals, two examples are the alveoli (air sacs) of the lungs (Chapter 3) and the villi of the small intestine (Chapter 4). Diffusion is a slow process, and organs that rely on diffusion need a large surface over which it can take place. The alveoli allow the exchange of oxygen and carbon dioxide to take place between the air and the blood during breathing. The villi of the small intestine provide a large surface area for the absorption of digested food. In plants, exchange surfaces are also adapted by having a large surface area, such as the spongy mesophyll of the leaf

In [7]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser


In [ ]:
# prompt for checking the LLM:
prompt = """
You are a topic relevancy checker. Your task is to determine if the user's question is relevant to Chapter: {chapterTitle} in {subject}.

Only respond with 'yes' if the question is related to the following chapter:
{chapterTitle}

Respond with 'no' if the question is:
- Off-topic or not related to this specific chapter
- Related to {subject} but from a different chapter
- Too general or vague to determine relevance
- About a completely different subject entirely

Question: {user_query}

Is this question relevant to Chapter {chapterTitle}?
Respond with only 'yes' or 'no'.
"""

student_input = "What is photosynthesis?"
llm_prompt = prompt.format(subject="Biology", chapterTitle="Cell Division", user_query=student_input)
model = OllamaLLM(model="llama3.2")
response = model(llm_prompt, output_parser=StrOutputParser())
response = response.lower().strip()

if "no" in response:
    print("The question is not relevant to Biology.")
if "yes" in response:
    a = rag.search(student_input, subject = "Biology", chapter= 17)

In [51]:
student_input = "What is photosynthesis?"
llm_prompt = prompt.format(subject="Biology", chapterTitle="Cell Division", user_query=student_input)

In [52]:
llm_prompt

"\nYou are a topic relevancy checker. Your task is to determine if the user's question is relevant to Chapter: Cell Division in Biology.\n\nOnly respond with 'yes' if the question is related to the following chapter:\nCell Division\n\nRespond with 'no' if the question is:\n- Off-topic or not related to this specific chapter\n- Related to Biology but from a different chapter\n- Too general or vague to determine relevance\n- About a completely different subject entirely\n\nQuestion: What is photosynthesis?\n\nIs this question relevant to Chapter Cell Division?\nRespond with only 'yes' or 'no'.\n"

In [53]:
model = OllamaLLM(model="llama3.2")

In [54]:
response = model(llm_prompt, output_parser=StrOutputParser())
response


'No.'

In [28]:
response = response.lower().strip()

In [55]:
if "no" in response:
    print("The question is not relevant to Biology.")

In [56]:
if "yes" in response:
    a = rag.search(student_input, subject = "Biology", chapter= 17)

In [57]:
a

"### KEY POINT\nOsmosis in cells is the net movement of water from a dilute solution to a more concentrated solution across the partially permeable cell membrane.\n\n## SPECIALISED EXCHANGE SURFACES\n\nAll cells exchange substances with their surroundings, but some parts of animals or plants are specially adapted for the exchange of materials because they have a very large surface area in proportion to their volume. In animals, two examples are the alveoli (air sacs) of the lungs (Chapter 3) and the villi of the small intestine (Chapter 4). Diffusion is a slow process, and organs that rely on diffusion need a large surface over which it can take place. The alveoli allow the exchange of oxygen and carbon dioxide to take place between the air and the blood during breathing. The villi of the small intestine provide a large surface area for the absorption of digested food. In plants, exchange surfaces are also adapted by having a large surface area, such as the spongy mesophyll of the leaf

In [15]:
print(a)

### KEY POINT
Osmosis in cells is the net movement of water from a dilute solution to a more concentrated solution across the partially permeable cell membrane.

## SPECIALISED EXCHANGE SURFACES

All cells exchange substances with their surroundings, but some parts of animals or plants are specially adapted for the exchange of materials because they have a very large surface area in proportion to their volume. In animals, two examples are the alveoli (air sacs) of the lungs (Chapter 3) and the villi of the small intestine (Chapter 4). Diffusion is a slow process, and organs that rely on diffusion need a large surface over which it can take place. The alveoli allow the exchange of oxygen and carbon dioxide to take place between the air and the blood during breathing. The villi of the small intestine provide a large surface area for the absorption of digested food. In plants, exchange surfaces are also adapted by having a large surface area, such as the spongy mesophyll of the leaf (Chap

In [7]:
rag.get_chapters_for_subject("Biology")

{1: 'Life Processes',
 2: 'The Variety of Living Organisms',
 3: 'Breathing and Gas Exchange',
 4: 'Food and Digestion',
 5: 'Blood and Circulation',
 6: 'Coordination',
 7: 'Chemical Coordination',
 8: 'Homeostasis and Excretion',
 9: 'Reproduction in Humans',
 10: 'Plants and Food',
 11: 'Transport in Plants',
 12: 'Chemical Coordination in Plants',
 13: 'Reproduction in Plants',
 14: 'Ecosystems',
 15: 'Human Influences on the Environment',
 16: 'Chromosomes, Genes and DNA',
 17: 'Cell Division',
 18: 'Genes and Inheritance',
 19: 'Natural Selection, Evolution and Selective Breeding',
 20: 'Using Microorganisms',
 21: 'Genetic Modification'}

In [8]:
rag.get_all_subjects()

['Biology']